In [3]:
import cv2
import yaml
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

import yaml

In [4]:
yaml_filename = "/home/kodogyu/kimera_multi_ws/src/kimera_vio/params/L515_stereo/FrontendParams.yaml"
left_camera_filename = "/home/kodogyu/kimera_multi_ws/src/kimera_vio/params/L515_stereo/LeftCameraParams.yaml"
right_camera_filename = "/home/kodogyu/kimera_multi_ws/src/kimera_vio/params/L515_stereo/RightCameraParams.yaml"

with open(yaml_filename, "r") as stream:
    stream.readline()  # %YAML:1.0 건너뛰기
    yaml_data = yaml.safe_load(stream)
print("yaml loaded")

with open(left_camera_filename, "r") as l_stream:
    l_stream.readline()  # %YAML:1.0 건너뛰기
    left_camera_params = yaml.safe_load(l_stream)
print("left camera parameters loaded")

with open(right_camera_filename, "r") as r_stream:
    r_stream.readline()  # %YAML:1.0 건너뛰기
    right_camera_params = yaml.safe_load(r_stream)
print("right camera parameters loaded")


yaml loaded
left camera parameters loaded
right camera parameters loaded


In [5]:
# calculate Left camera -> Right camera transformation matrix
body_T_Lcam = np.array(left_camera_params["T_BS"]["data"]).reshape((4, 4))
body_T_Rcam = np.array(right_camera_params["T_BS"]["data"]).reshape((4, 4))
# print("body_T_Lcam: ")
# print(body_T_Lcam)
# print("body_T_Rcam: ")
# print(body_T_Rcam)

l_rotation = body_T_Lcam[:-1, :-1]
l_translation = body_T_Lcam[:-1, 3].reshape((3, 1))
# r_rotation = body_T_Rcam[:-1, :-1]
# r_translation = body_T_Rcam[:-1, 3].reshape((3, 1))

# print("l_rotation: ")
# print(l_rotation)
# print("l_translation: ")
# print(l_translation)

l_inverse_rotation = l_rotation.transpose()
l_inverse_translation = -np.matmul(l_inverse_rotation, l_translation)
# r_inverse_rotation = r_rotation.transpose()
# r_inverse_translation = -np.matmul(r_inverse_rotation, r_translation)
Lcam_T_body = np.column_stack([l_inverse_rotation, l_inverse_translation])
Lcam_T_body = np.row_stack([Lcam_T_body, np.array([0, 0, 0, 1])])
# Rcam_T_body = np.column_stack([r_inverse_rotation, r_inverse_translation])
# Rcam_T_body = np.row_stack([Rcam_T_body, np.array([0, 0, 0, 1])])

camL_Pose_camR = np.matmul(body_T_Rcam, Lcam_T_body)
print("camL_Pose_camR: ")
print(camL_Pose_camR)

camL_Pose_camR: 
[[ 9.99987669e-01 -1.52341497e-04  4.96532807e-03 -8.94995766e-03]
 [ 1.51149190e-04  9.99999959e-01  2.37907917e-04 -4.77693889e-05]
 [-4.96536165e-03 -2.37162346e-04  9.99987642e-01  1.86356145e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
gtsam_pose_inv = np.array([[0.999988,  0.000246216,  -0.00495778,   0.00888662],
                    [-0.000247748, 1, -0.000305755,  0.000170101],
                    [0.00495771,  0.000306972,     0.999988, -0.000153022],
                    [0,            0,            0,            1]])

In [9]:
l_fx, l_fy, l_cx, l_cy = left_camera_params["intrinsics"]
r_fx, r_fy, r_cx, r_cy = right_camera_params["intrinsics"]
left_camera_intrinsic = np.array([[l_fx, 0, l_cx], [0, l_fy, l_cy], [0, 0, 1]])
right_camera_intrinsic = np.array([[r_fx, 0, r_cx], [0, r_fy, r_cy], [0, 0, 1]])

left_camera_distCoeffs = np.array(left_camera_params["distortion_coefficients"])
right_camera_distCoeffs = np.array(right_camera_params["distortion_coefficients"])

camL_Rot_camR = gtsam_pose_inv[:-1, :-1]
# camL_Tran_camR = -np.matmul(camL_Rot_camR, camL_Pose_camR[:-1, 3])
# camL_Rot_camR = camL_Pose_camR[:-1, :-1]
camL_Tran_camR = gtsam_pose_inv[:-1, 3]
kAlpha = 0

r1, r2, p1, p2, q, roi1, roi2= cv2.stereoRectify(left_camera_intrinsic, 
                                                 left_camera_distCoeffs, 
                                                 right_camera_intrinsic, 
                                                 right_camera_distCoeffs, 
                                                 np.array(left_camera_params["resolution"]), 
                                                 camL_Rot_camR, 
                                                 camL_Tran_camR, 
                                                 flags=cv2.CALIB_ZERO_DISPARITY, 
                                                 alpha=kAlpha
                                                 )

stereo_camera = p1
stereo_baseline = 1 / q[3, 2]
print("stereo_camera:")
print(stereo_camera)
print("stereo_baseline:")
print(stereo_baseline)

stereo_camera:
[[650.15163667   0.         336.90478134   0.        ]
 [  0.         650.15163667 261.60665894   0.        ]
 [  0.           0.           1.           0.        ]]
stereo_baseline:
-0.008889564956007972


In [11]:
print(left_camera_distCoeffs)
print(right_camera_distCoeffs)

[ 0.14669701 -0.27353153  0.00730068 -0.003734  ]
[ 0.14148691 -0.26292268  0.00708119 -0.00551033]
